In [404]:
# import classes from folder app/classes.py
from app.libs.classes import Shift, Nurse
import json


In [405]:
with open('data/shifts.json', 'r') as f:
    shifts = json.load(f)

shifts = {shift['shift_id']: Shift(**shift) for shift in shifts}


In [406]:
with open('data/nurses.json', 'r') as f:
    nurses = json.load(f)

nurses = {nurse['nurse_id']: Nurse(**nurse) for nurse in nurses}
nurses

{1: Nurse(nurse_id=1, nurse_name='Otalora_Castro_Rocio_del_Pilar', shift_preference='morning', accumulated_hours=-53.0, morning_availability_labor_day=1, morning_availability_weekend=1, afternoon_availability_labor_day=0, afternoon_availability_weekend=1, dates_off=[8, 14, 21, 22, 28], vacations=[]),
 2: Nurse(nurse_id=2, nurse_name='Lina_Mabel_Torres_Pulido', shift_preference='morning', accumulated_hours=-21.0, morning_availability_labor_day=1, morning_availability_weekend=1, afternoon_availability_labor_day=0, afternoon_availability_weekend=1, dates_off=[8, 15], vacations=[1, 2]),
 3: Nurse(nurse_id=3, nurse_name='Rodriguez_Bohorquez_Luz_Dary', shift_preference='afternoon', accumulated_hours=-33.5, morning_availability_labor_day=1, morning_availability_weekend=1, afternoon_availability_labor_day=0, afternoon_availability_weekend=1, dates_off=[8, 14, 21, 22, 28], vacations=[]),
 4: Nurse(nurse_id=4, nurse_name='Carrero_Contreras_Tania_Maryely', shift_preference='afternoon', accumulate

# Additional parameters 

In [407]:
DM = 8
HMM = 216

# Modelo

In [408]:
from datetime import datetime, timedelta
from math import floor
from typing import Dict

from app.libs.classes import Shift, Nurse
from app.libs.constants import HMM, DM
import pulp as plp
from itertools import product

In [409]:
import pulp as plp

model = plp.LpProblem("Nurse Scheduling", plp.LpMinimize)
I = set(nurses.keys())
J = set(shifts.keys())
K = set([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day for j in J])
week=set([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').isocalendar()[1] for j in J])

valid_keys = [(i, j) for (i,j) in product(I, J) if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day not in nurses[i].vacations]


/Users/user/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [410]:
X = plp.LpVariable.dicts("X", valid_keys, cat=plp.LpBinary)
W = plp.LpVariable.dicts("W", [(i, k) for (i,k) in product(I, K)], cat=plp.LpBinary)
Zmorning=plp.LpVariable.dicts("Zmorning", [(i, w) for (i,w) in product(I, week)], cat=plp.LpBinary)
Zafternoon=plp.LpVariable.dicts("Zafternoon", [(i, w) for (i,w) in product(I, week)], cat=plp.LpBinary)

V = plp.LpVariable.dicts("V", I, cat=plp.LpContinuous,lowBound=0)  # overtime
Y = plp.LpVariable.dicts("Y", I, cat=plp.LpContinuous,lowBound=0) # TOTAL NUMBER OF SHIFTS ASSIGNED TO NURSE i
MDH = plp.LpVariable("MDH", cat=plp.LpContinuous,lowBound=0) # MAXIMUM DIFFERENCE BETWEEN MAXIMUM SHIFTS AND REAL NUMBER OF SHIFTS ASSIGNED
DOff = plp.LpVariable.dicts("DOff", I, cat=plp.LpContinuous,lowBound=0) 


# Objective function

In [411]:
expr_PDL = plp.lpSum([(X[i, j] for (i,j) in valid_keys if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day in nurses[i].dates_off)])
# accounts for the number of weekends a person is assigned to a shif not in his/her preference
expr_PWE = plp.lpSum([(X[i, j]) for (i,j) in valid_keys if
                        datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').weekday() in [5, 6] and
                        (nurses[i].shift_preference != shifts[j].shift_type)])

Obj_2 = MDH + plp.lpSum([V[i] for i in I]) * (1 / DM)

In [412]:
# accounts the number of shifts assigned to a nurse
for nurse in I:
    model+=Y[nurse] == plp.lpSum([X[nurse, shift] for shift in J if (nurse, shift) in valid_keys]), f"assigned_shifts_{nurse}"

In [413]:
 # accounts for the difference between shifts that should be assigned to a caregiver 
            # (considering overtime hours to be balanced) and real number of shifts assigned

for nurse in I:
    expr = Y[nurse] - (HMM - nurses[nurse].accumulated_hours) * (1 / DM) <= MDH
    model += expr, f"balance_hours_1_{nurses[nurse].nurse_id}"

    expr = (HMM - nurses[nurse].accumulated_hours) * (1 / DM) - Y[nurse]<= MDH
    model += expr,  f"balance_hours_2_{nurses[nurse].nurse_id}"
    

In [414]:
# dictionary enumerating the shifts happening in each day
shifts_per_day = {}
for shift in J:
    if shifts[shift].shift_date not in shifts_per_day:
        shifts_per_day[shifts[shift].shift_date] = []
    shifts_per_day[shifts[shift].shift_date].append(shift)


In [415]:
for i in I:
    for spd in shifts_per_day.keys():
        model += plp.lpSum([X[i, j] for j in shifts_per_day[spd] if (i, j) in valid_keys]) <= 1, f"one_shift_per_day_{nurses[i].nurse_id}_{spd}"


In [416]:
# accounts for overtime V hours required
for i in I:
    model += Y[i] * DM <= (HMM - nurses[i].accumulated_hours) + V[i], f"overtime_{nurses[i].nurse_id}"

In [417]:
# demand is satisfied
for j in J:
    model += plp.lpSum([X[(i, j)] for i in I if (i, j) in valid_keys]) >= shifts[j].demand, f"demand_{j}"
        

In [418]:
# for each weekend, a nurse works at most one day
for i in I:
    for j1 in J:
        if (i, j1) in valid_keys and datetime.strptime(shifts[j1].shift_date,'%Y-%m-%d').weekday() == 5:
            model += X[i, j1] + plp.lpSum([X[i, j2] for j2 in J if
                                                        (i, j2) in valid_keys and datetime.strptime(shifts[j2].shift_date, '%Y-%m-%d').weekday() == 6
                                                        and datetime.strptime(shifts[j2].shift_date,'%Y-%m-%d') == datetime.strptime(shifts[j1].shift_date, '%Y-%m-%d') + timedelta(days=1)]) <= 1, f"weekend_{nurses[i].nurse_id}_{j1}"


In [419]:
for i,k in product(I, K):
    model += (1 - plp.lpSum([X[i, j] for j in J if (i, j) in valid_keys and datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day == k])) ==W[i, k], f"day_is_off_{nurses[i].nurse_id}_{k}"

In [420]:
days = list([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day for j in J if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').weekday() in [5, 6]])

total_weekend_days = len(days)

for i in I:
    model += DOff[i] == plp.lpSum([W[i, k] for k in days]), f"weekend_off_1_{nurses[i].nurse_id}"
    model += DOff[i] >= floor(total_weekend_days / 2) + 1, f"weekend_off_2_{nurses[i].nurse_id}"

In [421]:
for i,w in product(I, week):
    model += Zmorning[i, w] + Zafternoon[i, w] <= 1, f"shifts_per_week_{nurses[i].nurse_id}_{w}"

In [422]:
# if a nurse works in a morning shift the varriable Z_morning is equal to 1
for i in I:
    for j in J:
        if (i, j) in valid_keys and shifts[j].shift_type == 'morning':
            model +=  X[i, j]<=Zmorning[i, datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').isocalendar()[1]], f"morning_shift_{nurses[i].nurse_id}_{j}"
        elif (i, j) in valid_keys and shifts[j].shift_type == 'afternoon':
            model +=  X[i, j]<=Zafternoon[i, datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').isocalendar()[1]], f"afternoon_shift_{nurses[i].nurse_id}_{j}"

In [423]:
#an afternoon shift cannot be followed by a morning shift
for i in I:
    for j in J:
        if (i, j) in valid_keys and shifts[j].shift_type == 'morning':
            model +=  X[i, j] + plp.lpSum([X[i, j2] for j2 in J if
                                                        (i, j2) in valid_keys and datetime.strptime(shifts[j2].shift_date, '%Y-%m-%d') == datetime.strptime(shifts[j].shift_date, '%Y-%m-%d') - timedelta(days=1)
                                                        and shifts[j2].shift_type == 'afternoon']) <= 1, f"morning_afternoon_{nurses[i].nurse_id}_{j}"

In [424]:
#
model.setObjective(expr_PDL+expr_PWE)


In [425]:
# solve the problem using HIGHS
solver = plp.GUROBI_CMD(msg=0)
model.solve(solver)
#Obj_2 = MDH + plp.lpSum([V[i] for i in I]) * (1 / DM)


1

In [426]:
print(model.objective.value())

4.0


In [427]:

objective = model.objective.value()
epsilon = 1
model+= expr_PDL+expr_PWE<=objective + epsilon , "value_obj_1"
model.setObjective(Obj_2)
print(model.objective.value(),objective)
while model.status == 1:
    model.solve()
    print(model.objective.value(),objective + epsilon)
    epsilon += 1
    ctr = model.constraints["value_obj_1"]
    ctr.changeRHS(objective + epsilon)
    if objective + epsilon ==7:
        break
    
model.setObjective(Obj_2)

17.75 4.0
14.4375 5.0
13.4375 6.0


In [428]:
for i in I:
    for j in J:
        if (i, j) in valid_keys and X[i, j].varValue > 0.5:
            if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day in [1,7,8,14,15,21,22,28,29] and shifts[j].shift_type!=nurses[i].shift_preference:
                print(f"Nurse {nurses[i].nurse_name} works in shift {shifts[j].shift_id} on {shifts[j].shift_date} ({shifts[j].shift_type}--{shifts[j].shift}--{nurses[i].shift_preference}) with vacation {nurses[i].vacations} and days off {nurses[i].dates_off}.")
            if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day in nurses[i].dates_off:
                print(f"Nurse {nurses[i].nurse_name} works in shift {shifts[j].shift_id} on {shifts[j].shift_date} ({shifts[j].shift_type}--{shifts[j].shift}--{nurses[i].shift_preference}) with vacation {nurses[i].vacations} and days off {nurses[i].dates_off}.")

Nurse Rodriguez_Bohorquez_Luz_Dary works in shift 123 on 2022-05-21 (afternoon--T1--afternoon) with vacation [] and days off [8, 14, 21, 22, 28].
Nurse Carrero_Contreras_Tania_Maryely works in shift 3 on 2022-05-01 (afternoon--T1--afternoon) with vacation [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] and days off [1, 7, 8, 21, 29].
Nurse Carrero_Contreras_Tania_Maryely works in shift 39 on 2022-05-07 (afternoon--T1--afternoon) with vacation [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] and days off [1, 7, 8, 21, 29].
Nurse Ballesteros_Mesa_Jeimy_Catalina works in shift 44 on 2022-05-08 (morning--DISP--afternoon) with vacation [] and days off [1, 7, 14, 15, 29].
Nurse Ballesteros_Mesa_Jeimy_Catalina works in shift 126 on 2022-05-22 (morning--M--afternoon) with vacation [] and days off [1, 7, 14, 15, 29].
Nurse Murcia_Maira_Alejandra works in shift 162 on 2022-05-28 (morning--M--morning) with vacation [] and days off [1, 7, 15, 21, 28, 29].
